# OpenAI

In [4]:
# !pip install openai
from openai import OpenAI
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

get_response("What is the capital of France?")

'The capital of France is Paris.'

# LangChain


LangChain is an orchestration framework to build LLM applications leveraging a modularized infrastructure of chains and 
components like models, prompt templates and output parsers.

In [13]:
from IPython.display import Markdown
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

template = """Give me a list of 3 funny ideas for a company that makes {product}."""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

Markdown(chain.invoke({"product": "pancakes"}))

1. "Flippin' Fantastic Pancakes" - A pancake delivery service that guarantees perfectly flipped pancakes every time, with a comical twist on the traditional pancake flipping technique.

2. "Pancake Palooza" - A pancake restaurant that offers a variety of outrageous pancake toppings and flavors, such as bacon-infused pancake batter or rainbow sprinkle pancakes. Customers can also participate in pancake eating competitions for a chance to win free pancakes for life.

3. "Pancake Pals" - A subscription box service that delivers pancake mix and fun pancake accessories to your door every month. Each box includes a new pancake recipe and a surprise pancake-related gift, such as pancake molds in the shape of funny faces or animals.

# Pydantic

Pydantic is the most widely used data validation library for Python. With Pydantic, schema validation and serialization are controlled by type annotations; less to learn, less code to write, and integration with your IDE and static analysis tools.
Pydantic's core validation logic is written in Rust. As a result, Pydantic is among the fastest data validation libraries for Python. Pydantic models can emit JSON Schema, allowing for easy integration with other tools.

In [9]:
from datetime import datetime
from typing import Tuple
from pydantic import BaseModel


class Delivery(BaseModel):
    timestamp: datetime
    dimensions: Tuple[int, int]


m = Delivery(timestamp='2020-01-02T03:04:05Z', dimensions=['10', '20'])
print(repr(m.timestamp))
#> datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
print(m.dimensions)
#> (10, 20)

datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
(10, 20)


# Instructor

Instructor is a Python library that makes it a breeze to work with structured outputs from large language models (LLMs). Built on top of Pydantic, it provides a simple, transparent, and user-friendly API to manage validation, retries, and streaming responses.

In [11]:
# !pip install instructor
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI

# Patch the OpenAI client
client = instructor.from_openai(OpenAI())

class InvoiceData(BaseModel):
    """Call this function to extract invoice data with specific output structure"""
    invoice_number: str = Field(..., description="The invoice number extracted from the PDF")
    vat_registration_number: str = Field(..., description="The VAT registration number extracted from the PDF")

fake_invoice_data = """
Invoice Summary
==============
Invoice No: INV-123456
Date: 2024-02-11
Billing To: ACME Corporation
Address: 123 Business Rd, Business City, BC1234

Product Description       Quantity    Unit Price    Total
-----------------------------------------------------------
Widget A                  10          $15.00        $150.00
Gadget B                  5           $20.00        $100.00

Subtotal:                                          $250.00
Tax (10%):                                          $25.00
Total:                                             $275.00

VAT Registration Number: VAT-7890123

Thank you for your business!

"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
              {"role": "user", "content": fake_invoice_data}],
    response_model=InvoiceData
)

print(response.invoice_number)
print(response.vat_registration_number)

INV-123456
VAT-7890123


# DSPY

DSPy is a framework for algorithmically optimizing LM prompts and weights, especially when LMs are used one or more times within a pipeline. To use LMs to build a complex system without DSPy, you generally have to: (1) break the problem down into steps, (2) prompt your LM well until each step works well in isolation, (3) tweak the steps to work well together, (4) generate synthetic examples to tune each step, and (5) use these examples to finetune smaller LMs to cut costs. Currently, this is hard and messy: every time you change your pipeline, your LM, or your data, all prompts (or finetuning steps) may need to change.

In [ ]:
# !pip install dspy-ai

# Suppose you want to build a simple retrieval-augmented generation (RAG) system for question answering. 
# You can define your own RAG program like this:
import dspy

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        context = self.retrieve(question).passages
        answer = self.generate_answer(context=context, question=question)
        return answer

rag = RAG()  # zero-shot, uncompiled version of RAG
rag("what is the capital of France?").answer  # -> "Paris"

# Jinja2

Templating engine that allows you to write python-like code then pass data to render a document. 

In [12]:
# !pip install Jinja2
from jinja2 import Environment, PackageLoader, select_autoescape
# env = Environment(
#     loader=PackageLoader("yourapp"),
#     autoescape=select_autoescape()
# )
# template = env.get_template("mytemplate.html")
# print(template.render(the="variables", go="here"))
from jinja2.sandbox import SandboxedEnvironment
env = SandboxedEnvironment()
func = lambda: "Hello, Sandbox!"
print(env.from_string("{{ func() }}").render(func=func))

from openai import OpenAI
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content


get_response("What is the capital of Canada?")

func = lambda x: get_response(f"What is the capital of {x}?")
env.from_string("{{ func('Brazil') }}").render(func=func)

Hello, Sandbox!


'The capital of Brazil is Brasília. It was officially inaugurated as the capital on April 21, 1960.'

# Promptflow

[Promptflow](https://github.com/microsoft/promptflow) is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality. It is based on jinja!

# Guidance

Guidance is a framework for controlling large language models, it allows users to constrain generation (e.g. with regex and CFGs) as well as to interleave control (conditional, loops) and generation seamlessly.

it's good for:

1. Fine-grained controlled generation with open source models.
2. [Granular design for controlled llm generation in context lke rag search over web articles](https://github.com/guidance-ai/guidance/blob/7873d331e3f6b73cb5b6e2c1ed839377286d59d7/notebooks/art_of_prompt_design/rag.ipynb)
3. If statements within llm generation logic

In [9]:
# !pip install guidance

from guidance import models, gen, system, user, assistant

gpt = models.OpenAIChat("gpt-3.5-turbo-0125")

# append text or generations to the model
with system():
    lm = gpt + "You are a cat expert."

with user():
    lm += "What are the smallest cats?"

with assistant():
    lm += gen("answer", stop=".")

# GPT-Prompt-Engineer

It is a [github](https://github.com/mshumer/gpt-prompt-engineer) that introduced more structured experimentation
to prompt engineering, essentially allowing the user to generate, test and rank their prompts. 


# Marvin 

[Marvin](https://www.askmarvin.ai/docs/text/transformation/) is a framework that allows you to
to convert natural language to native Python types and structured objects. This is one of its simplest but most powerful features, and forms the basis for almost every other tool.

In [13]:
# !pip install marvin -U
import marvin
from pydantic import BaseModel

class Location(BaseModel):
    city: str
    state: str

marvin.cast("the big apple", target=Location)

Location(city='New York', state='New York')

In [15]:
with open("references.txt", "r") as f:
    references = f.read()
marvin.extract(references,
               instructions="Extract the urls from the references in the text above.")

['https://doi.org/10.1145/3491101.3519729',
 'https://doi.org/10.1145/3491102.3517582',
 'https://doi.org/10.1145/3487569',
 'https://doi.org/10.1098/rstb.2011.0416',
 'https://doi.org/10.1145/3490099.3511105',
 'https://doi.org/10.1145/3544548.3581388',
 'https://doi.org/10.1145/3543829.3543834',
 'https://doi.org/10.1016/j.iheduc.2019.03.002',
 'https://doi.org/10.1145/3520312.3534864',
 'https://doi.org/10.1080/03057267.2013.847261']

In [16]:
class URLs(BaseModel):
    urls: list

marvin.cast(references, target=URLs)

URLs(urls=['https://doi.org/10.1145/3491101.3519729', 'https://doi.org/10.1145/3491102.3517582', 'https://doi.org/10.1145/3487569', 'https://doi.org/10.1098/rstb.2011.0416', 'https://doi.org/10.1145/3490099.3511105', 'https://doi.org/10.1145/3544548.3581388', 'https://doi.org/10.1145/3543829.3543834', 'https://doi.org/10.1016/j.iheduc.2019.03.002', 'https://doi.org/10.1145/3520312.3534864', 'https://doi.org/10.1080/03057267.2013.847261'])

# Outlines

Outlines focuses on structured text generation, offering tools and functions for enhancing Large Language Model (LLM) prompting and generation processes. It provides a set of Python libraries and functions to facilitate complex text generation tasks.

For more information, visit [Outlines on GitHub](https://github.com/outlines-dev).

# LM-Studio

Awesome free interface to download and interact with open source LLMs including hosting your own server.

# LLM-CLI

LLM is a command-line tool and Python library for interacting with Large Language Models (LLMs), enabling users to run prompts, store results in SQLite, and generate embeddings. It supports both remote API models and self-hosted models through plugins, making it versatile for various applications.

For more information, visit [LLM on GitHub](https://github.com/simonw/llm).

# Fabric

Fabric is an open-source framework aimed at enhancing human capabilities using AI. It offers a modular approach to tackle specific problems through a crowdsourced collection of AI prompts that are versatile and applicable in various contexts.

For more details, visit [Fabric on GitHub](https://github.com/danielmiessler/fabric).

# SGLang

SGLang is a structured generation language specifically designed for enhancing interactions with large language models. It facilitates faster and more controlled communications by integrating a flexible frontend language with a high-performance runtime system, enabling advanced prompting techniques, parallelism, and external interactions. Similar to guidance and outlines.

For more information, visit [SGLang on GitHub](https://github.com/sgl-project/sglang).

# References
- https://huyenchip.com/2024/03/14/ai-oss.html?utm_source=bensbites&utm_medium=newsletter&utm_campaign=daily-digest-apple-and-amazon-are-waking-up